In [ ]:
from sqlalchemy import create_engine, text
import pandas as pd 
import numpy as np

In [ ]:
# ======================
# PostgreSQL Config
# ======================
DB_USER = "myuser"
DB_PASSWORD = "mypassword"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "mydatabase"

DATABASE_URL = (
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}"
    f"@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

engine = create_engine(DATABASE_URL)
print("✅ Connected to PostgreSQL")

✅ Connected to PostgreSQL


In [ ]:
def import_data(csv_path):
    df = pd.read_csv(csv_path)

    with engine.begin() as conn:
        # --- Lookup Tables ---
        # Gender
        genders = df['gender'].dropna().unique()
        for g in genders:
            conn.execute(
                text("INSERT INTO gender (gender_name) VALUES (:name) ON CONFLICT DO NOTHING"),
                {"name": str(g)}
            )

        # Branches
        branches = df['branch_no'].dropna().unique()
        for b in branches:
            conn.execute(
                text("INSERT INTO branches (branch_no) VALUES (:branch_no) ON CONFLICT DO NOTHING"),
                {"branch_no": int(b)}
            )

        # Careers
        careers = df['career'].dropna().unique()
        for c in careers:
            conn.execute(
                text("INSERT INTO careers (career_name) VALUES (:name) ON CONFLICT DO NOTHING"),
                {"name": str(c)}
            )

        # Provinces
        provinces = df['province'].dropna().unique()
        for p in provinces:
            conn.execute(
                text("INSERT INTO provinces (province_name) VALUES (:name) ON CONFLICT DO NOTHING"),
                {"name": str(p)}
            )

        # --- Insert Members and Addresses ---
        for _, row in df.iterrows():
            # Lookup FK ids and cast types
            gender_id = int(conn.execute(
                text("SELECT gender_id FROM gender WHERE gender_name=:name"),
                {"name": str(row['gender'])}
            ).scalar())

            branch_id = int(conn.execute(
                text("SELECT branch_id FROM branches WHERE branch_no=:branch_no"),
                {"branch_no": int(row['branch_no'])}
            ).scalar())

            career_id = conn.execute(
                text("SELECT career_id FROM careers WHERE career_name=:name"),
                {"name": str(row['career'])}
            ).scalar()
            career_id = int(career_id) if career_id is not None else None

            province_id = conn.execute(
                text("SELECT province_id FROM provinces WHERE province_name=:name"),
                {"name": str(row['province'])}
            ).scalar()
            province_id = int(province_id) if province_id is not None else None

            # Insert member
            conn.execute(text("""
                INSERT INTO members
                (member_id, first_name, last_name, gender_id, branch_id, birthday, registration_date, approval_date, career_id, income)
                VALUES
                (:member_id, :first_name, :last_name, :gender_id, :branch_id, :birthday, :registration_date, :approval_date, :career_id, :income)
            """), {
                "member_id": int(row['member_id']), # ดึง ID จาก CSV มาใช้เลย
                "first_name": str(row['first_name']),
                "last_name": str(row['last_name']),
                "gender_id": gender_id,
                "branch_id": branch_id,
                "birthday": row['birthday'],
                "registration_date": row['registration_date'],
                "approval_date": row.get('approval_date', None),
                "career_id": career_id,
                "income": float(row.get('income', 0))
            })

            # Get member_id
            current_member_id = int(row['member_id'])

            # Insert address
            conn.execute(text("""
                INSERT INTO addresses
                (member_id, house_no, moo, street, subdistrict, district, province_id, postal_code)
                VALUES
                (:member_id, :house_no, :moo, :street, :subdistrict, :district, :province_id, :postal_code)
            """), {
                "member_id": current_member_id,
                "house_no": str(row.get('house_no')),
                "moo": str(row.get('village_no')),      # ใน CSV ใช้ชื่อ village_no
                "street": str(row.get('road')),         # ใน CSV ใช้ชื่อ road
                "subdistrict": str(row.get('sub_area')), # ใน CSV ใช้ชื่อ sub_area
                "district": str(row.get('district_area')), # ใน CSV ใช้ชื่อ district_area
                "province_id": province_id,
                "postal_code": str(row.get('postal_code'))
            })

    print("✅ CSV data inserted successfully")

if __name__ == "__main__":
   import_data("/Users/hilmanyusoh/Desktop/Co-opDash/data/member300.csv")


✅ CSV data inserted successfully


# PRICE

In [ ]:
def import_data(csv_path):
    df = pd.read_csv(csv_path)

    with engine.begin() as conn:
        # --- Lookup Tables ---
        # Gender
        genders = df['table'].dropna().unique()
        for g in genders:
            conn.execute(
                text("INSERT INTO table (table_name) VALUES (:...) ON CONFLICT DO NOTHING"),
                {"...": str(g)}
            )  

        # --- Insert  ---
        for _, row in df.iterrows():
            # Lookup FK ids and cast types
            gender_id = int(conn.execute(
                text("SELECT ... FROM  WHERE ..."),
                {"name": str(row['gender'])}
            ).scalar())

            
            # Insert member
            conn.execute(text("""
                INSERT INTO table1
                (table1, )
                
                VALUES
                (:table1,)
            """), 

            )

            # Get member_id
            current_member_id = int(row['member_id'])


    print("✅ CSV data inserted successfully")

if __name__ == "__main__":
   import_data("/Users/hilmanyusoh/Desktop/Co-opDash/data/member300.csv")


✅ Connected to PostgreSQL
